# Convert Iceberg to Delta Lake

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from delta.tables import DeltaTable
import shutil
import random

testRoot = "/tmp/delta-iceberg-converter/"
warehousePath = testRoot + "iceberg_tables"
shutil.rmtree(testRoot, ignore_errors=True)

# we need to set the following configs
spark = SparkSession.builder \
    .appName("delta-iceberg-converter") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local.type", "hadoop") \
    .config("spark.sql.catalog.local.warehouse", warehousePath) \
    .getOrCreate()

table = "local.db.table"
tablePath = "file://" + warehousePath + "/db/table"

In [10]:
print("Creating Iceberg table with partitions...")
spark.sql(
    "CREATE TABLE {} (id BIGINT, data STRING) USING ICEBERG PARTITIONED BY (data)".format(table))
spark.sql("INSERT INTO {} VALUES (1, 'a'), (2, 'b')".format(table))
spark.sql("INSERT INTO {} VALUES (3, 'c')".format(table))